# Niave Bayes for Movie Reviews
* Make sure you have the file movie.csv in the same directory as this ipython notebook file

In [42]:
import pandas as pd #To prepare the data
import numpy as np #For the log function

### loading the data

In [43]:
#import the data using pandas
movie_data = pd.read_csv('movie.csv')
print("There are {} rows in the data set".format(len(movie_data)))
movie_data.head()

There are 2000 rows in the data set


,class,text
0,Pos,films adapted from comic books have had plenty...
1,Pos,every now and then a movie comes along from a ...
2,Pos,you ve got mail works alot better than it dese...
3,Pos,jaws is a rare film that grabs your attentio...
4,Pos,moviemaking is a lot like being the general ma...


### Divide the data into training and testing data

In [46]:
#First randomly select 20% of the data (400 rows)
movie_sample = movie_data.sample(400)
#The training data and the testing data MUST be seperated 
training_frame = movie_data[~movie_data.index.isin(movie_sample.index)]
testing_frame = movie_sample

### Define the dictionaries C, D

In [28]:
C = set(training_frame['class']) # Neg, Pos
D = dict()
for i in range(len(training_frame)):
    D[training_frame.iloc[i,1]] = training_frame.iloc[i,0]

### The Naive Bayes algorithm

In [30]:
def train_NB(C,D):
    V = set([word for doc in D.keys() for word in doc.split()])
    N = len(D)
    prior = dict()
    cond_prob = dict()
    N_c = dict()
    T = dict()
    text_c = dict()
    for c in C:
        text_c[c] = []
        for doc in D.items():
            if doc[1] == c:
                for word in doc[0].split():
                    text_c[c].append(word)
        N_c[c] = len([doc for doc in D.items() if doc[1] == c])
        prior[c] = float(N_c[c])/N
        cond_prob[c] = dict()
        T[c] = dict()
        for term in V:
            T[c][term] = text_c[c].count(term)
        for term in V:
            cond_prob[c][term] = float(T[c][term] + 1)/(sum(T[c].values()) + len(V))
 
    return V, prior, cond_prob

In [31]:
def test_NB(C,V,prior,cond_prob,d):
    W = []
    for word in d.split():
        if word in V:
            W.append(word)
    score = dict()
    for c in C:
        score[c] = np.log(prior[c])
        for term in W:
            score[c] += np.log(cond_prob[c][term])
    max_category = sorted(score.items(),key=lambda x: x[1],reverse= True)[0][0]
    return max_category

### Training
* on my computer this took about 10 minutes.  

In [33]:
V, prior, cond_prob = train_NB(C,D)

In [12]:
test_NB(C,V,prior,cond_prob,test_doc)

'china'

In [41]:
correct = 0
incorrect = 0
for i in range(len(testing_frame)): 
    if test_NB(C,V,prior,cond_prob,testing_frame.iloc[i,1]) == testing_frame.iloc[i,0]:
        correct += 1
    else:
        incorrect += 1
accuracy = correct/(correct + incorrect)
print("Accuracy = {}".format(accuracy))

Accuracy = 0.7975


# About 80% accuracy